Este trabalho testa três técnicas diferentes de classificação na base Adult, e ao final compara implementações e resultados.

Inicialização dos dados

In [ ]:
import pandas as pd
import numpy as np

adult = pd.read_csv("../input/adult-census-income/adult.csv", names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
adult_test = pd.read_csv("../input/us-census-data/adult-test.csv", names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
adult_training = pd.read_csv("../input/us-census-data/adult-training.csv", names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"],
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

# data prep

# remove first line
adult=adult[1:]
adult_test=adult_test[1:]

# remove missing data
nadult = adult.dropna()
nTestAdult = adult_test.dropna()
nTrainingAdult = adult_training.dropna()

Primeiro classificador testado: Random Forest

In [ ]:
# random forest

# data prep
from sklearn import preprocessing

numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
numTrainingAdult = nTrainingAdult.apply(preprocessing.LabelEncoder().fit_transform)

XtestAdult = numTestAdult.iloc[:,0:14]
YtestAdult = numTestAdult.Target
XtrainingAdult = numTrainingAdult.iloc[:,0:14]
YtrainingAdult = numTrainingAdult.Target

# train model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

rf = RandomForestRegressor(n_estimators = 400, random_state = 42)
rf.fit(XtrainingAdult, YtrainingAdult)
predictions = rf.predict(XtestAdult)

predictions = np.array(predictions)
YtestAdult = np.array(YtestAdult)

y = []
for i in predictions:
    if i >= 0.5:
        y.append(1)
    else:
        y.append(0)
y = np.array(y)
accuracy_score(YtestAdult, y)

Segundo classificador testado: Linear Regression

In [ ]:
# linear regression

# data prep
numAdult = nadult.apply(preprocessing.LabelEncoder().fit_transform)
numTestAdult = nTestAdult.apply(preprocessing.LabelEncoder().fit_transform)
numTrainingAdult = nTrainingAdult.apply(preprocessing.LabelEncoder().fit_transform)
Xadult = numAdult.iloc[:,0:14]
Yadult = numAdult.Target
XtestAdult = numTestAdult.iloc[:,0:14]
YtestAdult = numTestAdult.Target
XtrainingAdult = numTrainingAdult.iloc[:,0:14]
YtrainingAdult = numTrainingAdult.Target


YtestAdult = np.array(YtestAdult)

# train model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(XtrainingAdult, YtrainingAdult)
y_pred = regressor.predict(XtestAdult)

y = []
for i in y_pred:
    if i >= 0.5:
        y.append(1)
    else:
        y.append(0)

y = np.array(y)
accuracy_score(y, YtestAdult)

In [ ]:
#k-means
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

# sem seleção de atributos

# data prep
Xadult = numAdult.iloc[:,0:14]
Xadult = np.array(Xadult)
Yadult = numAdult.Target
Yadult = np.array(Yadult)

# model
km = KMeans(n_clusters = 2, init = 'random')
km.fit(Xadult)
y_km = km.predict(Xadult)
y_km = np.array(y_km)
a = accuracy_score(Yadult,y_km)
print(a)

# com seleção de atributos

#data prep
Xadult1 = nadult[["Age","Education-Num","Capital Gain", "Capital Loss", "Hours per week"]]
Yadult1 = numAdult.Target

# model
km1 = KMeans(n_clusters = 2, init = 'random')
km1.fit(Xadult1)
y1_km = km1.predict(Xadult1)
y1_km = np.array(y1_km)
b = accuracy_score(Yadult1,y1_km)
print(b)

**Considerações**

No trabalho, testou-se três técnicas de classificação na base Adult:
1. Random Forest
2. Linear Regression
3. K-Means
Sendo as duas primeiras técnicas de aprendizado supervisionado e a terceira de aprendizado não supervisionado.

Para a técnica de Random Forest:
O resultado obtido foi de 84.19% de acertos.
O pré processamento dos dados consistiu apenas em transformar os dados não numéricos em numéricos. 
O maior fator que influencia o tempo e uso de memória computacional é o número de árvores aleatórias geradas.
Nesse exemplo, foram usadas 400 árvores. Um outro teste feito anteriormente com 200 árvores alterou o resultado em menos de 0.1%.
Foi necessário transformar os valores de saída em valores binários. Para isso, usou-se a regra:
Se o valor é >= 0.5, é classificado como 1. Caso contrário, recebe valor 0.
O classificador foi criado com base no tutorial disponível em: 
https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

Para a técnica de Linear Regression:
O resultado obtido foi de 81.24% de acertos.
O classificador foi construído utilizando todos os atributos. Uma seleção dos atributos poderia ter melhorado o desempenho.
O pré processamento dos dados foi bem simples, apenas transformando todos os dados em valores numéricos.
O tempo computacional foi muito mais rápido que o classificador anterior.
Como as predições de uma regressão linear são valores contínuos, foi necessário transformá-los em binário: 
Possui valor 1 caso seu valor seja maior que 0.5, ou 0, caso contrário.
O classificador foi criado com base no tutorial disponível em: 
https://towardsdatascience.com/machine-learning-simple-linear-regression-with-python-f04ecfdadc13

Para a técnica de K-Means:
Foram feitos dois testes: sem seleção de atributos e com seleção de atributos.
Para o teste sem seleção de atributos:
O resultado obtido foi de aproximadamente 50% de acertos, uma taxa bem menor que os outros testes realizados.
Esse resultado pode variar aproximadamente 1% devido aos centros que são gerados aleatoriamente a cada vez que o programa é rodado.
Para o pré processamento dos dados foi necessário transformar os atributos não numéricos em numéricos.
Tentou-se criar dummies para cada atributo, mas a quantidade de variáveis fez o processamento ser inviável.
Como deve-se classificar o Target, foram usados 2 clusters.
Pode-se notar que o resultado foi bem abaixo do esperado, visto que houve influência a partir da transformação de dados não numéricos, e o classificador trabalhou apenas com os dados crus.
Um contorno para isso seria pré-selecionar os atributos. Para o teste feito com seleção:
O resultado obtido foi de aproximadamente 76%.
Pode-se notar uma clara melhora na classificação, mas ainda assim abaixo das outras duas testadas.
O classificador foi criado com base no tutorial disponível em: 
https://towardsdatascience.com/k-means-clustering-with-scikit-learn-6b47a369a83c